In [12]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-136902",subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f',
               resource_group='aml-quickstarts-136902')

exp = Experiment(workspace=ws, name="AutoMLProjExp")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136902
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-136902


In [13]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cpu_cluster_name = 'MLProjCompute'
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig


# Specify parameter sampler
ps = RandomParameterSampling( 
    {
        "--max_iter": choice(10,50,100,150,200)
        ,"--C": choice(0.5,0.8,0.9,1,1.25,1.5)
    }
)
        
# Specify a Policy
# Specify parameter sampler
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')
# src = ScriptRunConfig(source_directory='.', script='train.py', arguments=['--kernel', 'linear', '--penalty', 1.0], environment=sklearn_env)
src = ScriptRunConfig(source_directory='.', script='train.py', environment=sklearn_env)
src.run_config.target = compute_target

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=4)
print(hyperdrive_config)

In [15]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.core import Experiment, Workspace

# run = Experiment(ws,'train2').submit(src)
# run.wait_for_completion(show_output=True)

experiment = Experiment(ws, exp.name)
hyperdrive_run = experiment.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)
RunDetails(hyperdrive_run).show()

RunId: HD_9fc8f101-73db-4943-9228-4d9f6e161d3d
Web View: https://ml.azure.com/experiments/AutoMLProjExp/runs/HD_9fc8f101-73db-4943-9228-4d9f6e161d3d?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-136902/workspaces/quick-starts-ws-136902

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-02T00:04:42.664871][API][INFO]Experiment created<END>\n""<START>[2021-02-02T00:04:43.283008][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-02-02T00:04:43.5024808Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-02-02T00:04:43.578787][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_9fc8f101-73db-4943-9228-4d9f6e161d3d
Web View: https://ml.azure.com/experiments/AutoMLProjExp/runs/HD_9fc8f101-73db-4943-9228-4d9f6e161d3d?wsid=/subscriptions/a24a24d5-

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [16]:
import joblib
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

for child_run in hyperdrive_run.get_children_sorted_by_primary_metric():
    print(child_run)

print('\n')
print('############################################################')
print('\n')

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']


print('Best Run Id: ', best_run.id)
print('\n')
print('############################################################')
print('\n')
print("Best run metrics :",best_run.get_metrics())
print('\n')
print('############################################################')
print('\n')
print('Accuracy:', best_run_metrics['Accuracy'])
print('\n')
print('############################################################')
print('\n')
print("Best run details :",best_run.get_details())
print('\n')
print('############################################################')
print('\n')
print('Regularization Rate:',parameter_values)
print('\n')
print('############################################################')
print('\n')
print("Best run file names :",best_run.get_file_names())
print('\n')
print('############################################################')
print('\n')

# Register model
best_run.register_model(model_path='outputs/dpbanktrain.pkl', model_name='dpbanktrain',
                        tags={'Training context':'Hyperdrive'},
                        properties={'Accuracy': best_run_metrics['Accuracy']})



# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

{'run_id': 'HD_9fc8f101-73db-4943-9228-4d9f6e161d3d_14', 'hyperparameters': '{"--C": 1, "--max_iter": 200}', 'best_primary_metric': 0.9078907435508345, 'status': 'Completed'}
{'run_id': 'HD_9fc8f101-73db-4943-9228-4d9f6e161d3d_15', 'hyperparameters': '{"--C": 1.25, "--max_iter": 200}', 'best_primary_metric': 0.9072837632776934, 'status': 'Completed'}
{'run_id': 'HD_9fc8f101-73db-4943-9228-4d9f6e161d3d_5', 'hyperparameters': '{"--C": 0.9, "--max_iter": 200}', 'best_primary_metric': 0.9072837632776934, 'status': 'Completed'}
{'run_id': 'HD_9fc8f101-73db-4943-9228-4d9f6e161d3d_1', 'hyperparameters': '{"--C": 0.8, "--max_iter": 200}', 'best_primary_metric': 0.9072837632776934, 'status': 'Completed'}
{'run_id': 'HD_9fc8f101-73db-4943-9228-4d9f6e161d3d_11', 'hyperparameters': '{"--C": 0.9, "--max_iter": 150}', 'best_primary_metric': 0.9071320182094081, 'status': 'Completed'}
{'run_id': 'HD_9fc8f101-73db-4943-9228-4d9f6e161d3d_3', 'hyperparameters': '{"--C": 0.5, "--max_iter": 200}', 'best_pr

In [17]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_url_path)

In [18]:


from azureml.core import Dataset
import pandas as pd

def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    
    return x_df, y_df

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

ds_automl = Dataset.get_by_name(workspace=ws, name="Bank-marketing", version='latest')
train_data, test_data = ds_automl.random_split(percentage=0.5, seed=50)

In [19]:

from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5
)


In [20]:
# Submit your automl run

### YOUR CODE HERE ###

from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment = Experiment(ws, "AutoMLProjExp")
run_auto = experiment.submit(config=automl_config, show_output=True)

RunDetails(run_auto).show()


Running on remote.
No run_configuration provided, running on MLProjCompute with default configuration
Running on remote compute: MLProjCompute
Parent Run ID: AutoML_4394fdd3-29d2-4aa9-960b-e108e68f6381

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [21]:
best_run, fitted_model = run_auto.get_output()

print('############################################################')
print(best_run)
print('\n')
print('############################################################')
print('\n')
print(fitted_model)
print('############################################################')
print('\n')

# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
print('############################################################')
print('\n')

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print('############################################################')
print('\n')


############################################################
Run(Experiment: AutoMLProjExp,
Id: AutoML_4394fdd3-29d2-4aa9-960b-e108e68f6381_22,
Type: azureml.scriptrun,
Status: Completed)


############################################################


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_leaf=0.01,
                

In [22]:
best_run.get_metrics()

{'AUC_micro': 0.9803953108693932,
 'weighted_accuracy': 0.9575415778631478,
 'AUC_macro': 0.9463979700919809,
 'average_precision_score_micro': 0.9810183005886491,
 'precision_score_weighted': 0.9132305628956774,
 'average_precision_score_macro': 0.8237088311094117,
 'f1_score_macro': 0.7781726266920683,
 'precision_score_micro': 0.9181200647010581,
 'accuracy': 0.9181200647010581,
 'average_precision_score_weighted': 0.9549863618939195,
 'recall_score_weighted': 0.9181200647010581,
 'balanced_accuracy': 0.7596507669636692,
 'matthews_correlation': 0.5601656260134038,
 'f1_score_weighted': 0.9148945575479897,
 'AUC_weighted': 0.9463979700919809,
 'f1_score_micro': 0.9181200647010581,
 'recall_score_micro': 0.9181200647010581,
 'log_loss': 0.21678866636595878,
 'norm_macro_recall': 0.5193015339273382,
 'recall_score_macro': 0.7596507669636692,
 'precision_score_macro': 0.8026356619055252,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_4394fdd3-29d2-4aa9-960b-e108e68f6381

In [23]:
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [28]:
fitted_model.steps[1][1].estimators

[('1',
  Pipeline(memory=None,
           steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                  ('xgboostclassifier',
                   XGBoostClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1, missing=nan,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, tree_method='auto', verbose=-10,
                                     verbosity=0))],
           verbose=False)),
 ('0',
  Pipeline(memory=N

In [40]:
from pprint import pprint
def print_model(model, prefix=""):

    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
            e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
    
            for estimator in step[1].estimators:
                print('############################################################')
                print('\n')
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())

print()
print_model(fitted_model)



datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}
prefittedsoftvotingclassifier
{'estimators': ['1', '0', '19', '6', '20', '4', '5'],
 'weights': [0.25,
             0.08333333333333333,
             0.08333333333333333,
             0.16666666666666666,
             0.25,
             0.08333333333333333,
             0.08333333333333333]}

############################################################


1 - maxabsscaler
{'copy': True}
1 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 100,
 'n_job

In [41]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AutoMLProjExp,AutoML_4394fdd3-29d2-4aa9-960b-e108e68f6381_22,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [42]:
compute_target.delete()